In [1]:
import os
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

In [2]:
# Define labels
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
          'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'space', 'del']

In [3]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the image mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE)
landmarker = HandLandmarker.create_from_options(options)

In [5]:
# Loop through all the labels
for label_index, IMAGE_FILES in enumerate(labels):
    # Get all images in current label
    files = os.listdir(os.path.join('asl_data_right', IMAGE_FILES))
    
    # Total number of images
    size = len(files)
    
    # Initialize empty array to store landmarks
    arr = np.empty((size, 21, 2))
    
    # Loop through all images in each label
    for file_index, file in enumerate(files):
        # Load the input image.
        image = mp.Image.create_from_file(os.path.join('asl_data_right', IMAGE_FILES, file))

        # Detect hand landmarks from the input image.
        detection_result = landmarker.detect(image)
        
        # Get landmarks
        landmarks = detection_result.hand_world_landmarks[0]
        
        # Save landmarks to array
        for landmark_index, landmark in enumerate(landmarks):
            arr[file_index][landmark_index][0] = landmark.x
            arr[file_index][landmark_index][1] = landmark.y
            
    # Array to store label
    label_arr = np.tile(label_index, (size, 1))
    
    # Save arrays
    np.save('landmark_train_right/' + IMAGE_FILES + '-landmark', arr)
    np.save('landmark_train_right/' + IMAGE_FILES + '-label', label_arr)

In [6]:
# Initalize landmarks and labels array
landmark_train = []
label_train = []

# Loop through all the labels
for IMAGE_FILES in labels:
    # Load array and append
    landmark_train.append(np.load('landmark_train_right/' + IMAGE_FILES + '-landmark.npy'))
    label_train.append(np.load('landmark_train_right/' + IMAGE_FILES + '-label.npy'))
    
# Concatenate to a complete dataset
landmark_train = np.concatenate(landmark_train, axis=0)
label_train = np.concatenate(label_train, axis=0)

# Save dataset
np.save('landmark_train_right/landmark', landmark_train)
np.save('landmark_train_right/label', label_train)

In [8]:
print(np.load('landmark_train_right/landmark.npy').shape)
print(np.load('landmark_train_right/label.npy').shape)

(61711, 21, 2)
(61711, 1)
